# Batch mode script to call and run the pipeline

In [ ]:
# Install ipywidgets,ipyevents, circlefit and mayavi
!pip install ipywidgets
!pip install ipyevents 
!pip install mayavi
!pip install circle-fit

In [ ]:
# Import dependencies
import nibabel as nib
import numpy as np
import cv2
import scipy.io
import os
import imageio
import circle_fit as cf
from utils import euclidean_distance_coords
from tabulate import tabulate

# Create list of subfolders - names of patient files to run the batch mode on
subfolders = next(os.walk(os.path.join('data')))[1]
subfolders = ['DBS_bG56', 'DBS_bG64', 'DBS_bT20']
print(subfolders)

In [ ]:
# Load data, execute pipeline, and display outputs in batch for each inputted patient file name
data = []
for i in range(len(subfolders)):

    # Load patient data
    SAMPLE_NAME = subfolders[i]
    HULL_FILE_NAME = os.path.join('data/' + SAMPLE_NAME,"hull_rh.mat")
    PRECT_FILE_NAME = os.path.join('data/' + SAMPLE_NAME,"preop_ct.nii")
    POSTCT_FILE_NAME = os.path.join('data/' + SAMPLE_NAME,"postop_ct.nii")
    FLUORO_FILE_NAME = os.path.join('data/' + SAMPLE_NAME,"fluoro.tif")

    # read .mat File
    hull = scipy.io.loadmat(HULL_FILE_NAME)
    points_hull=np.array(hull['mask_indices'])

    #read pre op CT
    prect = nib.load(PRECT_FILE_NAME)
    prect_data = np.nan_to_num(np.array(prect.get_fdata()))
    Tmatrix = np.transpose(prect.affine)

    #read post op CT
    postct = nib.load(POSTCT_FILE_NAME)
    postct_data = np.nan_to_num(np.array(postct.get_fdata()))

    # fourth channel is infrared
    fluoro = imageio.imread(FLUORO_FILE_NAME)
    fluoro = cv2.cvtColor(fluoro, cv2.COLOR_BGR2GRAY)

    #Call main script to run all component scripts for this patient and display results to user
    %run Pipeline.ipynb
    
    #Load ground truth coordinates
    gt_dict = scipy.io.loadmat(os.path.join('data/' + SAMPLE_NAME + "/electrode_locations/electrode_locs.mat"))
    gt = gt_dict['CortElecLoc']
    gt = np.concatenate(gt,axis=0)
    new_gt = []
    for i in range (len(gt)):
        new = np.concatenate(gt[i],axis = 0)
        new_gt.append(new)
    new_gt = np.array(new_gt)
    new_gt = new_gt[0:len(predictions),:]
    
    #Append results from this patient
    data.append(SAMPLE_NAME)
    data.append(predictions)
    data.append(new_gt)
    data.append(np.mean(np.abs(new_gt - predictions)))
    data.append(euclidean_distance_coords(predictions, new_gt))
    
    #Display values of predicted and ground truth electrode coordinates
    print('The predicted ECoG (x,y,z) coordinates are')
    print(predictions)
    print('and the ground truth ECoG (x,y,z) coordinates are')
    print(new_gt)
    print('for patient')
    print(SAMPLE_NAME)

data = list(data)

CT segmentation visualization 2

###  DBS_bG56

In [18]:
from IPython.display import Image
Image(url="https://drive.google.com/uc?export=view&id=1H6GJcnixafTReEyp2a3ILdQ2NoY-ybgF")

###  DBS_bG64

In [19]:
Image(url="https://drive.google.com/uc?export=view&id=1BoxHu9nhSoy0TfDrxOMO6rIv4rezNuOR")

### DBSbT20

In [20]:
Image(url="https://drive.google.com/uc?export=view&id=10P3szX3SlnQ9UOII6OjdYEfI_Tsp4mru")

In [ ]:
# Print results of batch to a table
patients = [data[0], data[5], data[10]]
predict = [data[1], data[6], data[11]]
ground = [data[2], data[7], data[12]]
MAE = [data[3], data[8], data[13]]
EucDist = [data[4], data[9], data[14]]

table = zip(patients,predict,ground,MAE,EucDist)

headers=["Patient", "Predicted", "Ground Truth", "Mean Absolute Error", "Euclidean Distance"]

print(tabulate(table,headers))